In [801]:
import numpy as np
import pandas as pd
from math import sqrt,exp,pi,log
from random import randrange

In [802]:
df1 = pd.read_csv(r"C:\Users\shruthi sree\Downloads\hayes-roth.data",header = None)
df1.to_csv(r"C:\Users\shruthi sree\Desktop\ML_PA_1001933428\hayes-roth.csv",header=["name","hobby","age","educational-level","marital-status","class"])
df1 = pd.read_csv(r"C:\Users\shruthi sree\Desktop\ML_PA_1001933428\hayes-roth.csv")
data1 = df1.iloc[:,:].values

from sklearn.preprocessing import LabelEncoder  
lab = LabelEncoder()
for i in range(len(data1[0])):
    data1[:,i]=lab.fit_transform(data1[:,i])   
ulist = np.unique(data1[:,-1])


In [803]:
def eval_algo(data1, nfolds):
    folds = crossval_split(data1, nfolds)
    scores = list()
    i=0
    while i < (len(folds)):
        train_set=list()
        j=0
        while j< (len(folds)):
            if i!=j:
                train_set.append(folds[j])
            j+= 1
        train_set=list(folds)
        train_set = sum(train_set, [])
        test_set = list()
        for row in folds[i]:
            row1 = list(row)
            test_set.append(row1)
            row1[-1] = None
        pred = naive_bayes(train_set, test_set)
        act = [row[-1] for row in folds[i]]
        accuracy = acc_metric(act, pred)
        scores.append(accuracy)
        i+=1
    return scores

In [804]:
def crossval_split(data1, nfolds):
    data_split = list()
    data_copy = list(data1)
    fold_size = int(len(data1) / nfolds)
    for i in range(nfolds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(data_copy))
            fold.append(data_copy.pop(index))
        data_split.append(fold)
    return data_split

In [805]:
def naive_bayes(train, test):
    print(train)
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)

In [806]:
def summarize_by_class(data1):
    separated = split_by_class(data1,ulist)
    summaries = dict()
    for value, rows in separated.items():
        summaries[value] = summarize(rows)
    return summaries

In [807]:
def split_by_class(data1,ulist):
    slist=dict()
    for a in ulist:
        slist[a]=list()
    a=0
    while a < (len(data1)):
        vector=data1[a]
        value=vector[-1]
        slist[value].append(vector)
        a+=1
    return slist

In [808]:
def summarize(data1):
    summaries = [(mean(column), std_dev(column), len(column)) for column in zip(*data1)]
    del(summaries[-1])
    return summaries

In [809]:
def predict(summaries, row):
    prob = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for value, probability in prob.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = value
    return best_label

In [810]:
def calculate_probability(x, mean, stddev):
    if stddev==0.0 or mean==0.0:
        return 0.0
    exponent = exp(-((x-mean)**2 / (2 * stddev**2 )))
    return (1 / (sqrt(2 * pi) * stddev)) * exponent

In [811]:
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    prob = dict()
    for value, class_summaries in summaries.items():
        prob[value] = summaries[value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stddev, _ = class_summaries[i]
            prob[value] *= calculate_probability(row[i], mean, stddev)
    return prob

In [812]:
def mean(num):
    return sum(num)/float(len(num))
def std_dev(num):
    avg=mean(num)
    var = sum([(x-avg)**2 for x in num]) / float(len(num)-1) 
    return sqrt(var)

In [813]:
def acc_metric(act, pred):
    crct = 0
    a=0
    while a<(len(act)):
        if act[a] == pred[a]:
            crct += 1
        a+=1
    return crct / float(len(act)) * 100.0

In [814]:
scores=eval_algo(data1,10)
print('Scores: %s' % scores)
print('Mean Accuracy: %.4f%%' % (sum(scores)/float(len(scores))))

[array([ 8, 35,  1,  1,  0,  0,  0], dtype=int64), array([54, 95,  0,  0,  0,  1,  0], dtype=int64), array([47, 53,  0,  0,  0,  1,  0], dtype=int64), array([ 57, 122,   1,   0,   1,   0,   0], dtype=int64), array([40, 83,  1,  1,  1,  0,  1], dtype=int64), array([91, 64,  1,  2,  1,  2,  1], dtype=int64), array([ 70, 101,   2,   0,   3,   1,   2], dtype=int64), array([94, 58,  0,  0,  0,  1,  0], dtype=int64), array([108,  54,   0,   3,   1,   2,   2], dtype=int64), array([83, 34,  0,  1,  0,  2,  0], dtype=int64), array([131,   8,   2,   0,   0,   1,   0], dtype=int64), array([56, 69,  1,  1,  1,  0,  1], dtype=int64), array([118, 123,   2,   2,   1,   1,   1], dtype=int64), array([101,  96,   1,   0,   2,   0,   0], dtype=int64), array([109, 102,   1,   1,   0,   0,   0], dtype=int64), array([ 73, 118,   2,   0,   2,   1,   0], dtype=int64), array([38, 81,  0,  1,  0,  1,  1], dtype=int64), array([87, 30,  2,  2,  1,  0,  1], dtype=int64), array([84, 26,  0,  3,  3,  0,  2], dtype=i